In [1]:
from __future__ import print_function, division
import keras.backend as K
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import sklearn.metrics as metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Reshape, Concatenate, Layer, Dropout, RNN, LSTMCell, SimpleRNNCell,Bidirectional,LSTM,SimpleRNN
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, Flatten,LeakyReLU,ReLU,Lambda 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam
from functools import partial
from IPython.core.interactiveshell import InteractiveShell
from keras import metrics, losses
pd.options.display.max_rows = 2000

In [2]:
tf.compat.v1.disable_eager_execution()
gpu = tf.config.experimental.get_visible_devices('GPU')[0] # No GPU
tf.config.experimental.set_memory_growth(device = gpu, enable = True)

In [3]:
## Load the Population dataset
x_population = pd.read_csv('data/data_NHTSint_indiv.csv')
le = LabelEncoder()

# Note that [5,10] in Age is not the first factors 
for col in x_population.columns:
    x_population[col] = le.fit_transform(x_population[col])
x_population = x_population.apply(lambda x: x.astype('category'))


## Sample the 5% of samples
_,x_sample = train_test_split(x_population,test_size=0.05,shuffle=True,random_state=1004)

## Defining Catgroupsb
cat_groups = [0]
for col in x_population.columns:
    cat_groups.append(x_population[col].nunique())
    
cat_groups = np.cumsum(cat_groups)
cat_groups_n = len(cat_groups)-1

In [4]:
import sklearn

# Load pretrained bert model
mlm_model = tf.keras.models.load_model(
    "MLM_Embed_Indiv5_New.h5")
# mlm_model = tf.keras.models.load_model(
#     "MLM_Embed_Indiv5.h5")

embedding_layers =  mlm_model.layers[13:26] ## Find a embedding layers from mlm_model.summary()

def convert_to_embedding(samples):
    samples_emb = []
    for i in range(len(embedding_layers)):
        emb_weight = embedding_layers[i].get_weights()[0]
        trgt = samples[:,range(n_uni_col[i],n_uni_col[i+1])]
        samples_emb.append(np.dot(trgt,emb_weight))
    
    return(np.concatenate(samples_emb,axis=1))

def convert_to_embedding_tensor(y_pred):
    samples_emb = []
    for i in range(len(embedding_layers)):
        emb_weight = embedding_layers[i].get_weights()[0]
        trgt = y_pred[:,n_uni_col[i]:n_uni_col[i+1]]
        samples_emb.append(tf.tensordot(trgt,emb_weight,1))
    
    return(tf.concat(samples_emb,axis=1))

## Precision & Recall

def compute_pairwise_distance(data_x, data_y=None):

    if data_y is None:
        data_y = data_x
    dists = sklearn.metrics.pairwise_distances(
        data_x, data_y, metric='l2', n_jobs=-1)
    return dists

def compute_pairwise_distance_tensor(A,B):
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)
    # na as a row and nb as a co"lumn vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])
    
    # return pairwise euclidead difference matrix
    dists = tf.sqrt(tf.maximum(na - 2*tf.matmul(A, B, False, True) + nb, 1e-9))

    return dists

def compute_pairwise_hamming_distance(data_x, data_y=None):

    if data_y is None:
        data_y = data_x
    dists = sklearn.metrics.pairwise_distances(
        data_x, data_y, metric='hamming', n_jobs=-1)
    return dists

def compute_pairwise_hamming_distance_tensor(A,B):
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)
    # na as a row and nb as a column vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])
    
    # return pairwise euclidead difference matrix
    hamming_dists = na - 2*tf.matmul(A, B, False, True) + nb

    return hamming_dists

In [5]:
## Defining training hyperparameters
x_train = pd.get_dummies(train_test_split(x_population,test_size=0.05,shuffle=True,random_state=1004)[1])

from random import sample
## Make Ground Truth & Test
n_uni_col = [x_sample[i].nunique() for i in x_sample.columns]
n_uni_col = [0]+n_uni_col
n_uni_col = np.cumsum(n_uni_col)

col_pop = x_train.columns.values.copy()
for i in range(len(col_pop)):
    col_pop[i] = col_pop[i].split('_', 1)[1]
col_pop = col_pop.astype('int64')

In [6]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x_population)
x_sam_dum = enc.transform(x_sample).toarray()
sam_embed = convert_to_embedding(x_sam_dum)
sam_embed_tensor = tf.convert_to_tensor(sam_embed,dtype='float32')
x_sam_dum_tensor = tf.convert_to_tensor(x_sam_dum,dtype='float32')
#sam_embed_tensor = tf.constant(sam_embed,dtype='float32')
#sam_embed_tensor = tf.keras.initializers.Constant(sam_embed)

In [7]:
def build_generator():
    noise = Input(shape=(latent_dim))
    
    h = Dense(intermediate_dim[0],activation='relu')(noise)
    h = BatchNormalization()(h)
    for num_layer in range(len(intermediate_dim)-1):
        h = Dense(intermediate_dim[num_layer+1],activation='relu')(h)
        h = BatchNormalization()(h)

    cat_outputs = []
    for i in x_sample.columns:
        t = Dense(x_sample[i].nunique(),activation='softmax')(h)
        cat_outputs.append(t)                                
       
    concat = Concatenate()(cat_outputs)
    
    g_model = Model(noise,concat)

    return g_model
  

In [8]:
def build_critic():
    
    img = Input(shape=x_train.shape[1])
    
    h = Dense(intermediate_dim[0],activation=LeakyReLU(alpha=0.2))(img)
    for num_layer in range(len(intermediate_dim)-1):
        h = Dense(intermediate_dim[num_layer+1],activation=LeakyReLU(alpha=0.2))(h)

    validity = Dense(1)(h)
    
    c_model = Model(inputs = img,outputs = validity)

    return(c_model)

In [9]:
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def RandomWeightedAverage(inputs):
    alpha = K.random_uniform((BATCH_SIZE, 1))
    #alpha = tf.random.normal([BATCH_SIZE, 1],0.0,1.0)
    return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

        
def gradient_penalty_loss(y_true, y_pred, averaged_samples):
    """
    Computes gradient penalty based on prediction and weighted real / fake samples
    """
    gradients = K.gradients(y_pred, averaged_samples)[0]
    # compute the euclidean norm by squaring ...
    gradients_sqr = K.square(gradients)
    #   ... summing over the rows ...
    gradients_sqr_sum = K.sum(gradients_sqr,
                              axis=np.arange(1, len(gradients_sqr.shape)))
    #   ... and sqrt
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    # compute lambda * (1 - ||grad||)^2 still for each single sample
    gradient_penalty = K.square(1 - gradient_l2_norm)
    # return the mean as loss over all the batch samples
    return K.mean(gradient_penalty)

def OOS_loss(y_true,y_pred):
    # Get the OOS loss
    Hamming_distance = compute_pairwise_hamming_distance_tensor(y_pred,x_sam_dum_tensor)
    Hamming_nearest_distance = tf.math.reduce_min(Hamming_distance,axis=1)
    OOSloss = K.mean(Hamming_nearest_distance) 
    return OOSloss

def MS_loss(y_true,y_pred):
    # Get the OOS loss
    img1, img2 = tf.split(y_pred,2,axis=0,name='image_split')
    noise1, noise2 = tf.split(z_gen,2,axis=0,name='noise_split')
    MSloss = tf.reduce_mean(tf.abs(img1-img2)) / tf.reduce_mean(tf.abs(noise1-noise2))
    return -MSloss

# # div loss
def div_loss(y_true,y_pred):
  
    Hamming_distance = compute_pairwise_hamming_distance_tensor(y_pred,x_sam_dum_tensor)
    Hamming_mean_distance = tf.math.reduce_mean(Hamming_distance,axis=1)
    divloss = - K.mean(Hamming_mean_distance)
    
    return divloss

In [10]:
def wide_to_long(samples_pop):
    resamples = []
    for j in range(samples_pop.shape[0]):
        if(type(samples_pop) is np.ndarray):
            sam = samples_pop[j]
        else:
            sam = samples_pop.values[j]
        resamples_row = []
        for i in range(len(n_uni_col)-1):
            idx = range(n_uni_col[i],n_uni_col[i+1])
            resamples_row = np.append(resamples_row,np.random.choice(col_pop[idx],p=sam[idx],size=1))
        resamples = np.concatenate((resamples,resamples_row),axis=0)
    resamples = resamples.reshape(samples_pop.shape[0],len(n_uni_col)-1 )
    resamples = pd.DataFrame(resamples,columns= x_sample.columns.to_list())
    resamples = resamples.apply(lambda x: x.astype('category'))
    return(resamples)

def mean_JSD(samples,resamples):
    Marg_JSD = []
    for col in samples.columns:
        resam = pd.value_counts(resamples[col]).sort_index()
        sam = pd.value_counts(samples[col]).sort_index()
        tab = pd.merge(resam,sam,left_index=True, right_index=True,how='outer')
        tab = tab.fillna(0)
        Marg_JSD.append(jensenshannon(tab.iloc[:,0], tab.iloc[:,1]))
     
    bi_index = combinations(samples.columns,2)
    bi_index = list(bi_index)
    col1,col2 = bi_index[0]

    Bi_JSD = []
    for col1,col2 in bi_index:
        resam = pd.DataFrame(pd.crosstab(resamples[col1],resamples[col2],rownames=[col1],colnames=[col2]).stack().sort_index())
        sam = pd.DataFrame(pd.crosstab(samples[col1],samples[col2],rownames=[col1],colnames=[col2]).stack().sort_index())
        tab = pd.merge(resam,sam,left_index=True, right_index=True,how='outer')
        tab = tab.fillna(0)
        Bi_JSD.append(jensenshannon(tab.iloc[:,0], tab.iloc[:,1]))

    return([Marg_JSD,Bi_JSD])

def SRMSE(x_population,resamples):
    
    ## Aggregated Marginal distribution
    sam_marg_cnt = []
    resam_marg_cnt = []
    for col in x_population.columns:
        resam = pd.value_counts(resamples[col]).sort_index()
        sam = pd.value_counts(x_population[col]).sort_index()
        tab = pd.merge(resam,sam,left_index=True, right_index=True,how='outer')
        tab = tab.fillna(0)
        sam_marg_cnt.append(tab.iloc[:,1].values/x_population.shape[0])
        resam_marg_cnt.append(tab.iloc[:,0].values/resamples.shape[0])
    
    sam_marg_cnt = np.concatenate(sam_marg_cnt)
    resam_marg_cnt = np.concatenate(resam_marg_cnt)
    
    ## Aggregated Bivariate distribution    
    bi_index = combinations(x_population.columns,2)
    bi_index = list(bi_index)
    col1,col2 = bi_index[0]

    sam_bi_cnt  = []
    resam_bi_cnt = []
    for col1,col2 in bi_index:
        resam = pd.DataFrame(pd.crosstab(resamples[col1],resamples[col2],rownames=[col1],colnames=[col2]).stack().sort_index())
        sam = pd.DataFrame(pd.crosstab(x_population[col1],x_population[col2],rownames=[col1],colnames=[col2]).stack().sort_index())
        tab = pd.merge(resam,sam,left_index=True, right_index=True,how='outer')
        tab = tab.fillna(0)
        sam_bi_cnt.append(tab.iloc[:,1].values/x_population.shape[0])
        resam_bi_cnt.append(tab.iloc[:,0].values/resamples.shape[0])

    sam_bi_cnt = np.concatenate(sam_bi_cnt)
    resam_bi_cnt = np.concatenate(resam_bi_cnt)
    
    # SRMSE_mar
    rmse_mar = np.linalg.norm(sam_marg_cnt - resam_marg_cnt) / np.sqrt(len(sam_marg_cnt))
    ybar_mar = sam_marg_cnt.mean()
    srmse_mar = rmse_mar / ybar_mar

    # SRMSE_bi
    rmse_bi = np.linalg.norm(sam_bi_cnt - resam_bi_cnt) / np.sqrt(len(sam_bi_cnt))
    ybar_bi = sam_bi_cnt.mean()
    srmse_bi = rmse_bi / ybar_bi
    
    return([srmse_mar,srmse_bi])  

In [11]:
import time
start = time.time()

## Train
BATCH_SIZE = 256
epochs = np.int(200*x_train.shape[0]*0.7/BATCH_SIZE)
current_epochs= 1 # Epoch start from
sample_interval = 500 # Save checkpoint at every n epoch
n_critic = 3
losslog = []

C:\Users\euijin\AppData\Local\Temp\ipykernel_16740\2264731724.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  epochs = np.int(200*x_train.shape[0]*0.7/BATCH_SIZE)


In [12]:
import itertools
from random import sample
from scipy.spatial.distance import jensenshannon
from itertools import combinations

## Hyperparameter calibration
beta1_grid = [0.5,0.75]
beta2_grid = [0.004,0.005]
Performance_measures = []

param_grid = []
for r in itertools.product(beta1_grid, beta2_grid):
    param_grid.append([r[0],r[1]])

# Load the dataset
X_train = pd.get_dummies(train_test_split(x_population,test_size=0.05,shuffle=True,random_state=1004)[1]).values.astype("float32")

for beta in param_grid:
    
    # # Load the dataset
    # X_train = pd.get_dummies(train_test_split(x_population,test_size=0.05,shuffle=True,random_state=None)[1]).values.astype("float32")

    ## Setting hyperparameters from MultiCATGAN
    intermediate_dim = [256,256,256]
    latent_dim = 128
    optimizer = Adam(
        learning_rate=0.0002, beta_1=0.5, beta_2=0.9
    )
    BATCH_SIZE = 256
    beta1=beta[0]
    beta2=beta[1]

    ## Model Build
    generator = build_generator()
    critic = build_critic()

    #-------------------------------
    # Construct Computational Graph
    #       for the Critic
    #-------------------------------

    # Freeze generator's layers while training critic
    generator.trainable = False


    # Image input (real sample)
    real_img = Input(shape=x_train.shape[1])

    # Noise input
    z_disc = Input(shape=(latent_dim))
    # Generate image based of noise (fake sample) and add label to the input 
    fake_img = generator(z_disc)

    # Discriminator determines validity of the real and fake images
    fake = critic(fake_img)
    valid = critic(real_img)


    # Construct weighted average between real and fake images
    interpolated_img = RandomWeightedAverage([real_img, fake_img])

    # Determine validity of weighted sample
    validity_interpolated = critic(interpolated_img)

    partial_gp_loss = partial(gradient_penalty_loss,averaged_samples=interpolated_img)
    partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

    critic_model = Model(inputs=[real_img,z_disc], outputs=[valid, fake, validity_interpolated])
    critic_model.compile(loss=[wasserstein_loss,
                               wasserstein_loss,
                               partial_gp_loss],
                               optimizer=optimizer,
                               loss_weights=[1, 1, 10])




    #-------------------------------
    # Construct Computational Graph
    #         for Generator
    #-------------------------------

    # For the generator we freeze the critic's layers
    critic.trainable = False
    generator.trainable = True

    # Sampled noise for input to generator
    z_gen = Input(shape=(latent_dim))

    # Generate images based of noise
    img = generator(z_gen)

    # Discriminator determines validity
    valid = critic(img)

    # Defines generator model
    generator_model = Model(z_gen, [valid,img,img])
    generator_model.compile(loss=[wasserstein_loss,OOS_loss,div_loss], optimizer=optimizer,
                            loss_weights=[1,beta1,beta2])

    # Adversarial ground truths
    valid = - np.ones(BATCH_SIZE)
    fake =  np.ones(BATCH_SIZE)
    dummy = np.zeros(BATCH_SIZE) # Dummy gt for gradient penalty
    for epoch in range(epochs):
        for _ in range(n_critic):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            imgs = X_train[np.random.randint(0,X_train.shape[0],BATCH_SIZE)]

            # Sample generator input
            noise = np.random.normal(0,1,[BATCH_SIZE,latent_dim]) 

            # Train the critic
            d_loss = critic_model.train_on_batch([imgs, noise], [valid, fake, dummy])

        # ---------------------
        #  Train Generator
        # ---------------------
        g_loss = generator_model.train_on_batch(noise, [valid,valid,valid])


        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            print ("%d [D loss: %f] [G loss: %f] [OOS loss: %f] [Div loss: %f]" % (epoch, d_loss[0], g_loss[0], g_loss[1], g_loss[2]))
            losslog.append([d_loss[0], g_loss])
            generator.save_weights('C:/Users/euijin/Documents/modelCollection/WGAN/WGAN_OOSdivH'+str(beta), overwrite=True)
            #critic.save_weights('WGAN_Eager_critic/WGAN_Naive_Eager_F1', overwrite=True)
    
    print("time :", time.time() - start) 
    
    def generate_images(size=x_sample.shape[0]):
        generator.load_weights('C:/Users/euijin/Documents/modelCollection/WGAN/WGAN_OOSdivH'+str(beta))
        noise = np.random.normal(0, 1, (size,latent_dim))
        gen_imgs = generator.predict(noise)
   
        return gen_imgs


    ## Make Ground Truth & Test
    n_uni_col = [x_sample[i].nunique() for i in x_sample.columns]
    n_uni_col = [0]+n_uni_col
    n_uni_col = np.cumsum(n_uni_col)

    col_pop = x_train.columns.values.copy()
    for i in range(len(col_pop)):
        col_pop[i] = col_pop[i].split('_', 1)[1]
    col_pop = col_pop.astype('int64')
    
    ## Generate samples
    gen_sample = generate_images(x_sample.shape[0])
    resamples = wide_to_long(gen_sample)
    resamples = resamples.astype('int64')
    resamples = resamples.astype('category')
    
    ## Distributional distance
    Marg_JSD,Bi_JSD = mean_JSD(x_population,resamples)
    Marg_SRMSE,Bi_SRMSE = SRMSE(x_population,resamples)
    
    
    ## Extract the STZ/SAZ, Precision/Recall
    p_pop = x_population.iloc[:,0].astype(str)
    for i in range(x_population.shape[1]-1):
        p_pop = p_pop+x_population.iloc[:,(i+1)].astype(str)

    p_gen = resamples.astype('int64').iloc[:,0].astype(str)
    for i in range(resamples.astype('int64').shape[1]-1):
        p_gen = p_gen+resamples.astype('int64').iloc[:,(i+1)].astype(str)

    p_sam = x_sample.iloc[:,0].astype(str)
    for i in range(x_sample.shape[1]-1):
        p_sam = p_sam+x_sample.iloc[:,(i+1)].astype(str)
        
    STZ = resamples[~p_gen.isin(p_sam) & ~p_gen.isin(p_pop)]
    SAZ = resamples[~p_gen.isin(p_sam) & p_gen.isin(p_pop)]
    idx_STZ = ~p_gen.isin(p_sam) & ~p_gen.isin(p_pop)
    idx_SAZ = ~p_gen.isin(p_sam) & p_gen.isin(p_pop)
    STZ_ratio = round(STZ.shape[0]/p_gen.shape[0],3)
    SAZ_ratio = round(SAZ.shape[0]/p_gen.shape[0],3)
    Precision = round(np.mean(p_gen.isin(p_pop)),3)
    Recall = round(np.mean(p_pop.isin(p_gen)),3)
    Num_comb = p_gen.nunique()
    
    Performance_measures.append([round(Marg_SRMSE,3),round(Bi_SRMSE,3),Num_comb,Precision,Recall])
    

Instructions for updating:
Colocations handled automatically by placer.
0 [D loss: 3.349824] [G loss: 3.043435] [OOS loss: -0.077983] [Div loss: 6.338749]
500 [D loss: -0.991591] [G loss: 0.281251] [OOS loss: -0.288431] [Div loss: 1.244831]
1000 [D loss: -0.760539] [G loss: 0.401900] [OOS loss: -0.046122] [Div loss: 1.004580]
1500 [D loss: -0.607161] [G loss: 0.434072] [OOS loss: -0.008366] [Div loss: 0.994740]
2000 [D loss: -0.615086] [G loss: 0.425990] [OOS loss: 0.021933] [Div loss: 0.919477]
2500 [D loss: -0.496312] [G loss: 0.512406] [OOS loss: 0.173222] [Div loss: 0.791362]
3000 [D loss: -0.491020] [G loss: 0.611334] [OOS loss: 0.272835] [Div loss: 0.789873]
3500 [D loss: -0.515694] [G loss: 0.562926] [OOS loss: 0.234761] [Div loss: 0.769749]
4000 [D loss: -0.501835] [G loss: 0.514392] [OOS loss: 0.128378] [Div loss: 0.885595]
4500 [D loss: -0.503777] [G loss: 0.556013] [OOS loss: 0.200342] [Div loss: 0.825318]
5000 [D loss: -0.446996] [G loss: 0.517726] [OOS loss: 0.173276] [Div

C:\Users\euijin\.conda\envs\DGM\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


0 [D loss: 3.521802] [G loss: 3.062373] [OOS loss: -0.016044] [Div loss: 6.276444]
500 [D loss: -0.813097] [G loss: 0.173531] [OOS loss: -0.431148] [Div loss: 1.340166]
1000 [D loss: -0.704419] [G loss: 0.514697] [OOS loss: 0.041543] [Div loss: 1.081754]
1500 [D loss: -0.644966] [G loss: 0.726431] [OOS loss: 0.346714] [Div loss: 0.897425]
2000 [D loss: -0.709272] [G loss: 0.870713] [OOS loss: 0.475011] [Div loss: 0.930156]
2500 [D loss: -0.521706] [G loss: 0.847971] [OOS loss: 0.429199] [Div loss: 0.978607]
3000 [D loss: -0.552467] [G loss: 0.837238] [OOS loss: 0.481047] [Div loss: 0.853986]
3500 [D loss: -0.505551] [G loss: 0.761891] [OOS loss: 0.425222] [Div loss: 0.815237]
4000 [D loss: -0.469479] [G loss: 0.668537] [OOS loss: 0.303108] [Div loss: 0.873402]
4500 [D loss: -0.457403] [G loss: 0.657746] [OOS loss: 0.312088] [Div loss: 0.834133]
5000 [D loss: -0.372501] [G loss: 0.656981] [OOS loss: 0.305697] [Div loss: 0.846584]
5500 [D loss: -0.469975] [G loss: 0.622593] [OOS loss: 0.

C:\Users\euijin\.conda\envs\DGM\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


0 [D loss: 2.946622] [G loss: 4.806742] [OOS loss: 0.093409] [Div loss: 6.348338]
500 [D loss: -1.199839] [G loss: 1.201620] [OOS loss: 0.361425] [Div loss: 1.190309]
1000 [D loss: -1.051704] [G loss: 1.127373] [OOS loss: 0.557417] [Div loss: 0.832100]
1500 [D loss: -0.893421] [G loss: 1.025580] [OOS loss: 0.488384] [Div loss: 0.789358]
2000 [D loss: -0.802430] [G loss: 1.224481] [OOS loss: 0.700763] [Div loss: 0.771875]
2500 [D loss: -0.706849] [G loss: 1.061853] [OOS loss: 0.553000] [Div loss: 0.752186]
3000 [D loss: -0.769564] [G loss: 1.072818] [OOS loss: 0.539888] [Div loss: 0.785358]
3500 [D loss: -0.619707] [G loss: 1.006907] [OOS loss: 0.534437] [Div loss: 0.705391]
4000 [D loss: -0.725716] [G loss: 0.863187] [OOS loss: 0.427206] [Div loss: 0.656581]
4500 [D loss: -0.612588] [G loss: 0.937299] [OOS loss: 0.503779] [Div loss: 0.653595]
5000 [D loss: -0.586758] [G loss: 0.959576] [OOS loss: 0.487236] [Div loss: 0.705259]
5500 [D loss: -0.574939] [G loss: 0.924026] [OOS loss: 0.45

C:\Users\euijin\.conda\envs\DGM\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


0 [D loss: 3.622234] [G loss: 4.717385] [OOS loss: -0.001377] [Div loss: 6.372538]
500 [D loss: -1.114619] [G loss: 0.901132] [OOS loss: -0.008108] [Div loss: 1.300929]
1000 [D loss: -0.941767] [G loss: 0.983563] [OOS loss: 0.370092] [Div loss: 0.907670]
1500 [D loss: -0.826401] [G loss: 1.023832] [OOS loss: 0.478131] [Div loss: 0.818905]
2000 [D loss: -0.845680] [G loss: 1.118835] [OOS loss: 0.600885] [Div loss: 0.782559]
2500 [D loss: -0.732065] [G loss: 1.094143] [OOS loss: 0.615484] [Div loss: 0.731789]
3000 [D loss: -0.801053] [G loss: 1.180424] [OOS loss: 0.705166] [Div loss: 0.727015]
3500 [D loss: -0.680214] [G loss: 1.060416] [OOS loss: 0.646449] [Div loss: 0.646586]
4000 [D loss: -0.581393] [G loss: 1.148587] [OOS loss: 0.679803] [Div loss: 0.719464]
4500 [D loss: -0.701635] [G loss: 1.011102] [OOS loss: 0.639688] [Div loss: 0.590493]
5000 [D loss: -0.688979] [G loss: 1.045122] [OOS loss: 0.632597] [Div loss: 0.645104]
5500 [D loss: -0.650278] [G loss: 1.061838] [OOS loss: 0.

C:\Users\euijin\.conda\envs\DGM\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [14]:
Outputs = pd.concat([pd.DataFrame(param_grid),pd.DataFrame(np.concatenate(Performance_measures).reshape(-1,5))],axis=1)
Outputs.to_clipboard()


In [ ]:
from random import sample
## Make Ground Truth & Test
n_uni_col = [x_sample[i].nunique() for i in x_sample.columns]
n_uni_col = [0]+n_uni_col
n_uni_col = np.cumsum(n_uni_col)

col_pop = x_train.columns.values.copy()
for i in range(len(col_pop)):b
    col_pop[i] = col_pop[i].split('_', 1)[1]
col_pop = col_pop.astype('int64')


In [ ]:
gen_sample = generate_images(x_sample.shape[0])
resamples = wide_to_long(gen_sample)

In [ ]:

Marg_JSD,Bi_JSD = mean_JSD(x_sample,resamples)
print(np.mean(Marg_JSD),np.mean(Bi_JSD))
Marg_JSD,Bi_JSD = mean_JSD(x_population,resamples)
print(np.mean(Marg_JSD),np.mean(Bi_JSD))

In [ ]:
## Extract the STZ and SAZ
p_pop = x_population.iloc[:,0].astype(str)
for i in range(x_population.shape[1]-1):
    p_pop = p_pop+x_population.iloc[:,(i+1)].astype(str)
    
p_gen = resamples.astype('int64').iloc[:,0].astype(str)
for i in range(resamples.astype('int64').shape[1]-1):
    p_gen = p_gen+resamples.astype('int64').iloc[:,(i+1)].astype(str)
    
p_sam = x_sample.iloc[:,0].astype(str)
for i in range(x_sample.shape[1]-1):
    p_sam = p_sam+x_sample.iloc[:,(i+1)].astype(str)

In [ ]:
print({'out-of-popgen': round(1-np.sum(p_gen.isin(p_pop))/len(p_gen),3),
       'out-of-samgen': round(1-np.sum(p_gen.isin(p_sam))/len(p_gen),3)})

In [ ]:
STZ = resamples[~p_gen.isin(p_sam) & ~p_gen.isin(p_pop)]
SAZ = resamples[~p_gen.isin(p_sam) & p_gen.isin(p_pop)]
idx_STZ = ~p_gen.isin(p_sam) & ~p_gen.isin(p_pop)
idx_SAZ = ~p_gen.isin(p_sam) & p_gen.isin(p_pop)
print(round(STZ.shape[0]/p_gen.shape[0],3),round(SAZ.shape[0]/p_gen.shape[0],3))

In [ ]:
print(np.mean(p_pop.isin(p_sam)))
print(np.mean(p_pop.isin(p_gen)))

In [ ]:
Precision = np.mean(p_gen.isin(p_pop))
Recall = np.mean(p_pop.isin(p_gen))

In [ ]:
Precision = []
Recall = []
for num in range(430000,1130000,100000):
    t_gen_sample = generate_images(num)
    t_resamples = wide_to_long(t_gen_sample)
    t_p_gen = t_resamples.astype('int64').iloc[:,0].astype(str)
    for i in range(t_resamples.astype('int64').shape[1]-1):
        t_p_gen = t_p_gen+t_resamples.astype('int64').iloc[:,(i+1)].astype(str)
    Precision.append(np.mean(t_p_gen.isin(p_pop)))
    Recall.append(np.mean(p_pop.isin(t_p_gen)))
    print(num); print(Precision); print(Recall)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x_population)

x_pop_dum = enc.transform(x_population).toarray()
x_gen_dum = enc.transform(resamples).toarray()
x_sam_dum = enc.transform(x_sample).toarray()

In [ ]:
## Precision & Recall
import numpy as np
from prdc import compute_prdc
from sklearn import preprocessing
from sklearn.neighbors import BallTree 
import sklearn.metrics
from scipy.spatial.distance import cdist

def compute_nearest_neighbour_distances(input_features, nearest_k):

    tree = BallTree(input_features,leaf_size = 40,metric='l1')
    dist, _ = tree.query(input_features, k=(nearest_k+1)) 
    radii = dist
    return radii

def compute_pairwise_distance(data_x, data_y=None):

    if data_y is None:
        data_y = data_x
    dists = sklearn.metrics.pairwise_distances(
        data_x, data_y, metric='l2', n_jobs=-1)
    return dists

def compute_pairwise_hamming_distance(data_x, data_y=None):

    if data_y is None:
        data_y = data_x
    dists = sklearn.metrics.pairwise_distances(
        data_x, data_y, metric='hamming', n_jobs=-1)
    return dists

def compute_pairwise_nearest_distance(data_x, data_y):

    kd_tree1 = cKDTree(data_x)
    kd_tree2 = cKDTree(data_y)
    r = np.max(compute_nearest_neighbour_distances(
    data_x, nearest_k))
    indexes = kd_tree1.query_ball_tree(kd_tree2, r=0.2,p=1)
    
    
    dists = sklearn.metrics.pairwise_distances(
        data_x, data_y, metric='l1', n_jobs=-1)
    return dists

In [ ]:
# Load pretrained bert model
mlm_model = tf.keras.models.load_model(
    "MLM_Embed_Indiv5_New.h5")
# mlm_model = tf.keras.models.load_model(
#     "MLM_Embed_Indiv5.h5")

embedding_layers =  mlm_model.layers[13:26] ## Find a embedding layers from mlm_model.summary()

def convert_to_embedding(samples):
    samples_emb = []
    for i in range(len(embedding_layers)):
        emb_weight = embedding_layers[i].get_weights()[0]
        trgt = samples[:,range(n_uni_col[i],n_uni_col[i+1])]
        samples_emb.append(np.dot(trgt,emb_weight))
    
    return(np.concatenate(samples_emb,axis=1))

In [ ]:
sam_embed = convert_to_embedding(x_sam_dum)
#nearest_k = 3
#real_nearest_neighbour_distances = compute_nearest_neighbour_distances(sam_embed, nearest_k) ## 2 minutes

In [ ]:
STZ_dum = enc.transform(STZ).toarray()
STZ_embed = convert_to_embedding(STZ_dum)
SAZ_dum = enc.transform(SAZ).toarray()
SAZ_embed = convert_to_embedding(SAZ_dum)

In [ ]:
STZ_distance = compute_pairwise_distance(STZ_embed,sam_embed)
STZ_nearest_distance = np.min(STZ_distance,axis=1)

SAZ_distance = compute_pairwise_distance(SAZ_embed,sam_embed)
SAZ_nearest_distance = np.min(SAZ_distance,axis=1)

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as st
%matplotlib inline

def plot_histogram(x,y):
    # q25, q75 = np.percentile(y, [25, 75])
    # bin_width = 2 * (q75 - q25) * len(x) ** (-1/3)
    # bins = round((x.max() - x.min()) / bin_width)
    bins = 50
    
    plt.hist(x,bins=bins,density=True,label='STZ',alpha=0.5)
    plt.hist(y,bins=bins,density=True,label='SAZ',alpha=0.5)
    
    mn, mx = plt.xlim()
    plt.xlim(mn, mx)
    kde_xs = np.linspace(mn, mx, 300)
    kde = st.gaussian_kde(x)
    plt.plot(kde_xs, kde.pdf(kde_xs), label="STZ_PDF")

    mn, mx = plt.xlim()
    plt.xlim(mn, mx)
    kde_xs = np.linspace(mn, mx, 300)
    kde = st.gaussian_kde(y)
    plt.plot(kde_xs, kde.pdf(kde_xs), label="SAZ_PDF")
    

    plt.legend(loc="upper right")
    plt.ylabel("Probability")
    plt.xlabel("Nearest distance from sample")
    print("Freedman–Diaconis number of bins:", bins)
    plt.show()

plot_histogram(STZ_nearest_distance,SAZ_nearest_distance)

In [ ]:
## Compute hamming distance of STZ and SAZ
from scipy.spatial.distance import hamming

STZ_hamming_distance = compute_pairwise_hamming_distance(STZ_dum,x_sam_dum)
SAZ_hamming_distance = compute_pairwise_hamming_distance(SAZ_dum,x_sam_dum)
STZ_hamming_distance  = np.min(STZ_hamming_distance ,axis=1)
SAZ_hamming_distance  = np.min(SAZ_hamming_distance ,axis=1)

In [ ]:
## Construct logistic regression model
from sklearn.linear_model import LogisticRegression
data_distance_to_manifold = pd.DataFrame({'DTM': np.hstack([STZ_nearest_distance,SAZ_nearest_distance]),
                                          'label': np.hstack([np.repeat(1,len(STZ_nearest_distance)),np.repeat(2,len(SAZ_nearest_distance))])},
                                         index =np.arange(0,(len(STZ_nearest_distance)+len(SAZ_nearest_distance))))

X = np.array(data_distance_to_manifold.DTM).reshape(-1,1)
y = np.array(data_distance_to_manifold.label)
clf = LogisticRegression(random_state=0).fit(X, y)
clf.score(X, y)

In [ ]:
## Construct logistic regression model
from sklearn.linear_model import LogisticRegression

data_distance_to_manifold = pd.DataFrame({'DTM': np.hstack([STZ_hamming_distance,SAZ_hamming_distance]),
                                          'label': np.hstack([np.repeat(1,len(STZ_hamming_distance)),np.repeat(2,len(SAZ_hamming_distance))])},
                                         index =np.arange(0,(len(STZ_hamming_distance)+len(SAZ_hamming_distance))))

X = np.array(data_distance_to_manifold.DTM).reshape(-1,1)
y = np.array(data_distance_to_manifold.label)
clf = LogisticRegression(random_state=0).fit(X, y)
clf.score(X, y)

In [ ]:
## Random selection
(STZ.shape[0]/(STZ.shape[0]+SAZ.shape[0]))**2 + (SAZ.shape[0]/(STZ.shape[0]+SAZ.shape[0]))**2

In [ ]:
def compute_prdc(real_features_sam,fake_features_sam,nearest_k):
    
    print('Num real: {} Num fake: {}'
      .format(real_features_sam.shape[0], fake_features_sam.shape[0]))
    
    real_nearest_neighbour_distances = compute_nearest_neighbour_distances(
        real_features_sam, nearest_k)
    fake_nearest_neighbour_distances = compute_nearest_neighbour_distances(
        fake_features_sam, nearest_k)
    distance_real_fake = compute_pairwise_distance(
        real_features_sam, fake_features_sam)

    precision = (
            distance_real_fake <
            np.expand_dims(real_nearest_neighbour_distances, axis=1)
    ).any(axis=0).mean()

    recall = (
            distance_real_fake <
            np.expand_dims(fake_nearest_neighbour_distances, axis=0)
    ).any(axis=1).mean()

    density = (1. / float(nearest_k)) * (
            distance_real_fake <
            np.expand_dims(real_nearest_neighbour_distances, axis=1)
    ).sum(axis=0).mean()

    coverage = (
            distance_real_fake.min(axis=1) <
            real_nearest_neighbour_distances
    ).mean()


    return np.array([round(precision,3),round(recall,3),round(density,3), round(coverage,3)])


In [ ]:
# Load pretrained bert model
mlm_model = tf.keras.models.load_model(
    "MLM_Embed_Indiv5_New.h5")
# mlm_model = tf.keras.models.load_model(
#     "MLM_Embed_Indiv5.h5")

embedding_layers =  mlm_model.layers[13:26] ## Find a embedding layers from mlm_model.summary()

def convert_to_embedding(samples):
    samples_emb = []
    for i in range(len(embedding_layers)):
        emb_weight = embedding_layers[i].get_weights()[0]
        trgt = samples[:,range(n_uni_col[i],n_uni_col[i+1])]
        samples_emb.append(np.dot(trgt,emb_weight))
    
    return(np.concatenate(samples_emb,axis=1))

In [ ]:
## Precision & Recall
import numpy as np
from prdc import compute_prdc
from sklearn import preprocessing
from sklearn.neighbors import BallTree 
import sklearn.metrics
from scipy.spatial.distance import cdist

def compute_nearest_neighbour_distances(input_features, nearest_k):

    tree = BallTree(input_features,leaf_size = 40,metric='l1')
    dist, _ = tree.query(input_features, k=(nearest_k+1)) 
    radii = dist[:,nearest_k]
    return radii

def compute_pairwise_distance(data_x, data_y=None):

    if data_y is None:
        data_y = data_x
    dists = sklearn.metrics.pairwise_distances(
        data_x, data_y, metric='l1', n_jobs=-1)
    return dists


# def compute_pairwise_distance(data_x, data_y=None):

#     if data_y is None:
#         data_y = data_x
#     dists = cdist(data_x, data_y, metric='euclidean')
#     return dists

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x_population)

x_pop_dum = enc.transform(x_population).toarray()
x_gen_dum = enc.transform(resamples).toarray()
x_sam_dum = enc.transform(x_sample).toarray()

In [ ]:
import time
st = time.time()

pop_embed = convert_to_embedding(x_pop_dum)
gen_embed = convert_to_embedding(x_gen_dum)
sam_embed = convert_to_embedding(x_sam_dum)

pop_features = pop_embed[np.random.choice(pop_embed.shape[0], size=100000, replace=False)]
gen_features = gen_embed[np.random.choice(gen_embed.shape[0], size=5000, replace=False)]
sam_features = sam_embed[np.random.choice(sam_embed.shape[0], size=5000, replace=False)]

results =[]
for k in [3,5,7]:
    metrics_gen = compute_prdc(pop_features,gen_features,k)
    metrics_sam = compute_prdc(pop_features,sam_features,k)
    results.append(np.concatenate([metrics_gen,metrics_sam],axis=0))
    
outputs = pd.DataFrame(np.concatenate(np.array(results),axis=0).reshape(-1,8))
outputs.to_clipboard()

et = time.time()
print(et-st)

In [ ]:
STZ = []
SAZ = []


np.sum(p_gen.isin(p_pop))/len(p_gen)


In [ ]:
p_sam.nunique()

In [ ]:
import json


### SC data

samples_train =  wide_to_long(x_train_cond)

## OOS Ratio
resamples_pvalue = resamples_SC.iloc[:,0].astype(str)
for i in range(resamples_SC.shape[1]-1):
    resamples_pvalue = resamples_pvalue+resamples_SC.iloc[:,(i+1)].astype(str)
    
samples_pvalue = samples.iloc[:,0].astype(str)
for i in range(samples.shape[1]-1):
    samples_pvalue = samples_pvalue+samples.iloc[:,(i+1)].astype(str)

resamples_pvalue_ind = resamples_SC.iloc[:,0].astype(str)
for i in range(resamples_SC.shape[1]-6):
    resamples_pvalue_ind  = resamples_pvalue_ind +resamples_SC.iloc[:,(i+1)].astype(str)
    
samples_pvalue_ind = samples.iloc[:,0].astype(str)
for i in range(samples.shape[1]-6):
    samples_pvalue_ind  = samples_pvalue_ind +samples.iloc[:,(i+1)].astype(str)
    
    
OOS_resam = resamples_SC.shape[0]-np.sum(resamples_pvalue.isin(samples_pvalue))
OOS_ratio = OOS_resam/resamples_SC.shape[0]
OOS_resam_ind = resamples_SC.shape[0]-np.sum(resamples_pvalue_ind.isin(samples_pvalue_ind))
OOSI_ratio_ind = OOS_resam_ind/resamples_SC.shape[0]


## CZ Ratio
C1 = sum(resamples_SC['TP_0'] == 'TP_0_Z')
C2 = sum((resamples_SC['TP_1'] == 'TP_1_Z'))
C3 = sum((resamples_SC['TP_2'] == 'TP_2_Z') & ((resamples_SC['TP_3'] != 'TP_3_Z') | (resamples_SC['TP_4'] != 'TP_4_Z')))
C4 = sum((resamples_SC['TP_3'] == 'TP_3_Z') & ((resamples_SC['TP_4'] != 'TP_4_Z')))
CZ = C1+C2+C3+C4
print(OOS_ratio)
print(OOSI_ratio_ind)
print(CZ/OOS_resam)

